## ENPM 809F Final Project
## MQTT Protocol Simulation for Home Automation System
## Muhammad Adil Sohail & Waleed Waris
## 12/14/2020

* This project script will create an IoT simulation which will use MQTT protocol to simulate an IoT network in a home automation system. The simulation will leverage a dataset that provides actual data from sensors in a home automation system such as temperature, humidity, light on/off, etc. Data from the dataset will be read, published to a broker, and recieved by a smartphone application that is subscribed to the proper messages. In addition, commands for certain actuators can be sent from the smartphone application.

* This is one of the two modules for the project. The other being Client_subscriber. This module will read the sensor data, and publish the sensor data to the proper topic. In addition, this module will publish the command data from the mobile_device application to the respective topic. 

* A GUI will be leveraged to display the sensor data / actuator commands

## 1.0 Load Sensor Dataset

In [1]:
# Import Neccesary Libraries 
import paho.mqtt.client as mqtt
import time
import pandas as pd
import os 
import copy
from tkinter import *
import time
import datetime

In [2]:
# The portion of the code below will import the dataset, analyze the dataset,
# and transform it to be used in the MQTT Protocol Simulation

path = os.getcwd() # Get working directory 
dataset_path = os.path.join(path,'IoT_SensorReading_Dataframe.csv')
df=pd.read_csv(dataset_path) #read sensor dataset 
df # Print df dataset

,ts,device,co,humidity,light,lpg,motion,smoke,temp
0,1.594512e+09,b8:27:eb:bf:9d:51,0.004956,51.000000,False,0.007651,False,0.020411,22.700000
1,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.700001
2,1.594512e+09,b8:27:eb:bf:9d:51,0.004976,50.900000,False,0.007673,False,0.020475,22.600000
3,1.594512e+09,1c:bf:ce:15:ec:4d,0.004403,76.800003,True,0.007023,False,0.018628,27.000000
4,1.594512e+09,b8:27:eb:bf:9d:51,0.004967,50.900000,False,0.007664,False,0.020448,22.600000
...,...,...,...,...,...,...,...,...,...
405179,1.595203e+09,00:0f:00:70:91:0a,0.003745,75.300003,False,0.006247,False,0.016437,19.200001
405180,1.595203e+09,b8:27:eb:bf:9d:51,0.005882,48.500000,False,0.008660,False,0.023301,22.200000
405181,1.595203e+09,1c:bf:ce:15:ec:4d,0.004540,75.699997,True,0.007181,False,0.019076,26.600000
405182,1.595203e+09,00:0f:00:70:91:0a,0.003745,75.300003,False,0.006247,False,0.016437,19.200001


In [3]:
df_new=df.loc[df["device"]=="b8:27:eb:bf:9d:51",:] #Narrows dataset for data in living room only (focus of our project)

df_new_copy = copy.deepcopy(df_new) # Creates copy of dataset so we can map
df_new_copy["ts"] = pd.to_datetime(df_new['ts'], unit='s')
df_new_copy["light"] = df_new["light"].map({True: "On",False: "Off"}) #Map light to On/Off
df_new_copy["motion"] = df_new["motion"].map({True: "Yes",False: "No"}) #Map motion to Yes/No
df_new = df_new_copy 
df_new = df_new.round(decimals=9)

df_new=df_new[:1000] # Very large dataset, change this to control simulaton length.

x=[x for x in range (0,len(df_new))]
s = pd.Series(x)
df_new.set_index(s,inplace=True)
df_new.head(5) # Print df_new dataset

,ts,device,co,humidity,light,lpg,motion,smoke,temp
0,2020-07-12 00:01:34.385969877,b8:27:eb:bf:9d:51,0.004956,51.0,Off,0.007651,No,0.020411,22.7
1,2020-07-12 00:01:38.073570013,b8:27:eb:bf:9d:51,0.004976,50.9,Off,0.007673,No,0.020475,22.6
2,2020-07-12 00:01:41.761229992,b8:27:eb:bf:9d:51,0.004967,50.9,Off,0.007664,No,0.020448,22.6
3,2020-07-12 00:01:45.448859930,b8:27:eb:bf:9d:51,0.004976,50.9,Off,0.007673,No,0.020475,22.6
4,2020-07-12 00:01:49.136680126,b8:27:eb:bf:9d:51,0.004970,50.9,Off,0.007667,No,0.020457,22.6


In [4]:
df_new.info() #Describes dataset category / columns etc. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ts        1000 non-null   datetime64[ns]
 1   device    1000 non-null   object        
 2   co        1000 non-null   float64       
 3   humidity  1000 non-null   float64       
 4   light     1000 non-null   object        
 5   lpg       1000 non-null   float64       
 6   motion    1000 non-null   object        
 7   smoke     1000 non-null   float64       
 8   temp      1000 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 78.1+ KB


In [5]:
df_new.describe() #Conducts high level stats for numerical dataset with describe function

,co,humidity,lpg,smoke,temp
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.005015,50.637800,0.007717,0.020600,22.447100
std,0.000031,0.803054,0.000034,0.000097,0.212595
min,0.004914,48.700000,0.007604,0.020276,22.000000
25%,0.004995,50.500000,0.007694,0.020534,22.300000
50%,0.005015,51.100000,0.007717,0.020599,22.500000
75%,0.005038,51.100000,0.007742,0.020673,22.600000
max,0.005120,51.500000,0.007833,0.020931,22.700000


In [6]:
# We will work with this dataset to show our 
# understanding of message exchange between multiple clients and single MQTT broker/server

## 2.0 Define & Connect Clients and Broker 

In [7]:
# Defines all sensors as clients below

client1=mqtt.Client(client_id="CO_Sensor")
client2=mqtt.Client(client_id="humidity_Sensor")
client3=mqtt.Client(client_id="light_Sensor")
client4=mqtt.Client(client_id="lpg_Sensor")
client5=mqtt.Client(client_id="motion_Sensor")
client6=mqtt.Client(client_id="smoke_Sensor")
client7=mqtt.Client(client_id="temp_Sensor")

# Defines actuators / switches as clients

client8=mqtt.Client(client_id="temp_Controller")
client9=mqtt.Client(client_id="light_Controller")
client10=mqtt.Client(client_id="fan_Controller")

#Defines mobile application as client

client_mobile_device=mqtt.Client(client_id="mobile_device")
clientClock=mqtt.Client(client_id="Clock")

In [8]:
# Defines local/external broker 

broker_external="test.mosquitto.org"
broker_local="localhost"

# Connects clients to local server 
rc1=client1.connect(broker_local,port=1883)
rc2=client2.connect(broker_local,port=1883)
rc3=client3.connect(broker_local,port=1883)
rc4=client4.connect(broker_local,port=1883)
rc5=client5.connect(broker_local,port=1883)
rc6=client6.connect(broker_local,port=1883)
rc7=client7.connect(broker_local,port=1883)
rc8=client8.connect(broker_local,port=1883)
rc9=client9.connect(broker_local,port=1883)
rc10=client10.connect(broker_local,port=1883)
rc11=client_mobile_device.connect(broker_local,port=1883)
clientClock.connect(broker_local,port=1883)

#Checks if all clients are connected to server

if (rc1==0 and rc2==0 and rc3==0 and rc4==0 and rc5==0 and rc6==0 and rc7==0 and rc8==0 and rc9==0 and rc10==0 and rc11==0):
    ConCheck = 1
    print("Connection Established")
else:
    ConCheck = 0
    print("Connection Error")

Connection Established


## 3.0 Publish Messages to Broker

In [9]:
# The following code will iterate through the sensor dataset and publish data 
# for each client to the proper channel

# Following will publish commands from mobile device to different devices 
client_mobile_device.publish("house/livingroom/temp/set","100", retain=True)
client_mobile_device.publish("house/livingroom/Light","ON", retain=True)
client_mobile_device.publish("house/livingroom/Fan","ON", retain=True)

# The following loop will publish sensor data from clients to server on proper channel 

for [ind,row] in df_new.iterrows(): #iterates through sensor dataframe
    if (ind%10)==0:
        print(ind)

        clientClock.publish("house/livingroom/time",str(row[0])) #Prints timestamp
        client1.publish("house/livingroom/CO",str(row[2]))
        client2.publish("house/livingroom/humidity",str(row[3]))
        client3.publish("house/livingroom/lightdetected",str(row[4]))
        client4.publish("house/livingroom/lpg",str(row[5]))
        client5.publish("house/livingroom/motiondetected",str(row[6]))
        client6.publish("house/livingroom/smoke",str(row[7]))  
        client7.publish("house/livingroom/temp",str(row[8]))

        time.sleep(2)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
